La empresa La Jara Metales es una compañía especializada en aleaciones específicas de la industria aérea. La Junta de Extremadura les ha concedido un contrato para proveer 2000 kg de una aleación a 105 euros/kg. La aleación debe de contener Cobre, Magnesio, Niquel y Aluminio. Los requisitos de composición que se deben cumplir son:
1. Al menos un 20% de Níquel 
2. Al menos un 15% de Cobre
3. Entre un 2% y un 3% de Magnesio
4. No más del 1,5% de impurezas
5. El resto Aluminio.

La empresa dispone de 5 metales que pueden mezclarse para fabricar el producto final. Los costes y composición de estos se encuentran en la hoja Datos/Datos_Ejercicio29.xlsx. El material Z20 es muy demandado, por lo que la empresa no puede utilizar más de 600 kg para fabricar la aleación. La empresa está interesada en conocer qué mezcla de materiales básicos proporciona los mayores beneficios. También están interesados en conocer los cambios en los valores de algunos parámetros. 
1. ¿Cómo afectaría a la decisión final disminuir el coste de Z30 en 5 eu/kg?
2. ¿Cuánto afectaría una restricción muy severa de la legislación en cuanto al contenido de impurezas?
3. Si la disponibilidad de Z20 pudiera aumentarse pagando 10 eur adicionales por tonelada, ¿sería interesante pagar este coste adicional?

In [83]:
from pyomo.environ import *
import pandas as pd
model = ConcreteModel()

In [84]:
datos = pd.read_excel('Datos_Ejercicio29.xlsx', index_col=0)
model.z= Set(initialize=datos.index.to_list())

In [85]:
model.F=Var(model.z, within=NonNegativeReals)

model.Cu = Constraint(expr=sum(model.F[z]*datos['%Cu'][z] for z in model.z)>=sum(model.F[z] for z in model.z)*15)
model.Mgmin = Constraint(expr=sum(model.F[z]*datos['%Mg'][z] for z in model.z)>=sum(model.F[z] for z in model.z)*2)
model.Mgmax = Constraint(expr=sum(model.F[z]*datos['%Mg'][z] for z in model.z)<=sum(model.F[z] for z in model.z)*3)
model.Ni = Constraint(expr=sum(model.F[z]*datos['%Ni'][z] for z in model.z)>=sum(model.F[z] for z in model.z)*20)
model.imp = Constraint(expr=sum(model.F[z]*datos['%imp'][z] for z in model.z)<=sum(model.F[z] for z in model.z)*1.5)
model.Tot= Constraint(expr=sum(model.F[z] for z in model.z)==2000)
model.Z20= Constraint(expr=model.F['Z20']<=600)

model.obj=Objective(expr=2000*105-sum(model.F[z]*datos['Coste_eu/kg'][z] for z in model.z), sense=maximize)


In [86]:
results=SolverFactory('glpk').solve(model)
results.write()
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 77800.0
  Upper bound: 77800.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 6
  Number of nonzeros: 27
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.08029365539550781
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

Disminuir el coste de Z30 no afecta a la solución del problema

El mínimo contenido en impurezas afecta mucho. Por debajo del 1% el problema no tendría solución. Fijándolo en ese valor la FO pasa de 77800 a 27800.

Si el coste de Z20 aumenta 10 eur/kg y se elimina la restricción de los 600 kg máximo, la nueva solución del problema es 73000 menor que en el caso anterior